In [73]:
import os 
import pandas as pd 
import json 
from tqdm import tqdm 
import collections 
import random

In [26]:
# xsum_dir_path = "../src/data/cliff_multiple_summaries"

# Json to CSV

In [27]:
xsum_dir_path = "../src/data/cliff_multiple_summaries"

# xsum_topics = [8, 17, 22, 20, 118, 58, 145, 127, 100, 114, 132, 79]
# xsum_topics = [112, 148, 104, 61, 71, 27, 2, 0, 34, 64, 52, 139, 76, 74, 50, 63, 98]
xsum_topics = [30, 75, 46, 94, 116, 149, 84, 126, 6, 109, 37, 141, 135, 3, 4]

In [28]:
factscore_dir_path = "../src/data/factscore_filtered"
factscore_topics = [x[:-5] for x in os.listdir(factscore_dir_path) if x.endswith("json")]


In [29]:
tofueval_mediasum = "../src/data/tofueval_chosen/mediasum"
mediasum_topics = [x[:-5] for x in os.listdir(tofueval_mediasum) if x.endswith("json")]

In [44]:
verifiability_path = "../src/data/verifiability"
# verifiability_topics = [x[:-5] for x in os.listdir(verifiability_path) if x.endswith("json")]
verifiability_topics = ['608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b87363ffb7c4dbccfe5b75-perplexity',
                 '608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b87363ffb7c4dbccfe5b75-neeva',
 '5f70135447f685d2cfb0b6e41b00f2ac787f50611adfde29dd60e9d10a1df831-perplexity',
 '9befc3bbc7fe5614a69a577553195d42cb7f9468e74d996337a4e67993effa44-perplexity',
 '9befc3bbc7fe5614a69a577553195d42cb7f9468e74d996337a4e67993effa44-neeva',
 'b4f9e9e26e0fd56b0bd5899ce85c75299d6dc5a89c8ce4878bab2c98e752376f-perplexity',
 'b4f9e9e26e0fd56b0bd5899ce85c75299d6dc5a89c8ce4878bab2c98e752376f-neeva',
 'deae1a7412a8eb9c480322daaed12652bdb50d789d09a393a93348350a4febc1-perplexity',
 'ebb3be87f4929f116372109e6e2f166d5ce21d378f61eee6db33e8b306a96ff5-neeva',
 '308bc8aafa9e96c8ef2c67a01024067171ef6ba93e6479c0d2caaabf50a0fc4b-perplexity',
 'affec3fdd0e64a10e1ebf080072586eb4ab38b6f1449502ebf4ba839fd434429-neeva',
 '1dfb92efea40d15b76af074cf37b9b990d50feb10efdaac6636d788f2b0b0e6b-neeva',
 'afa4a5de3563cabb30e68722419064cf4ce932e0cb5024d2bb96b3991f2399f9-perplexity',
'67247a4db1b0891fc049618dee77f53ebf388b9b97aacaa9bfd41864de874853-perplexity']

In [45]:
len(verifiability_topics)

14

In [64]:
all_qas = pd.DataFrame()
for example_topic in verifiability_topics: #factscore_topics[10:]: # xsum_topics:
    with open(os.path.join(verifiability_path, f"{example_topic}.json"), "r") as f:
        data = json.load(f)
    for i, summary in enumerate(data["summaries"]):
        df_tokens = pd.DataFrame(summary["tokens"])
        sentence_mapping = df_tokens[["sent_id", "text"]].groupby("sent_id")["text"].apply(lambda group: " ".join(group)).to_dict()

        df = pd.DataFrame(summary["qas"])

        df["sentence"] = df["sentId"].apply(lambda x: sentence_mapping[x])
        df["source_id"] = data["sourceId"]
        df["summary_id"] = i
        df["model"] = summary["summaryId"]
        df["origin_id"] = df["source_id"].apply(lambda x: x.split("-")[0])
        all_qas = pd.concat([all_qas, df[["origin_id", "source_id", "summary_id", "model", "questionId", "sentId", "sentence", "predicate", "question", "answer"]]])


In [65]:
summary.keys()

dict_keys(['tokens', 'spans', 'qas', 'predicates', 'label', 'summaryId'])

In [66]:
df_tokens[["sent_id", "text"]].groupby("sent_id")["text"].apply(lambda group: " ".join(group)).to_dict()

{0: 'It is also recommended to stay hydrated and avoid caffeine .'}

In [67]:
df.head()

,sentId,predicateId,predicate,predicatePos,question,answer,answerStartToken,answerEndToken,answerId,cleanAnswerId,verbTokenId,sourceIds,questionId,sentence,source_id,summary_id,model,origin_id
0,0,3-4,recommended,VERB,what is recommended?,It,[0],[1],0-1,0-1,2,"[129, 261, 777, 142, 399, 16, 1173, 406, 918, ...",0,It is also recommended to stay hydrated and av...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,3,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...
1,0,3-4,recommended,VERB,what is recommended?,to stay hydrated and avoid caffeine,[4],[10],4-10,5-10,2,"[771, 712, 713, 232, 847, 848, 657, 306, 307, ...",1,It is also recommended to stay hydrated and av...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,3,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...
2,0,5-6,stay,VERB,why should someone stay?,It is also recommended,[0],[4],0-4,0-4,3,"[129, 643, 261, 777, 142, 399, 16, 1173, 406, ...",2,It is also recommended to stay hydrated and av...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,3,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...
3,0,5-6,stay,VERB,how should someone stay?,hydrated,[6],[7],6-7,6-7,3,"[848, 771]",3,It is also recommended to stay hydrated and av...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,3,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...
4,0,8-9,avoid,VERB,what should be avoided?,caffeine,[9],[10],9-10,9-10,2,"[712, 657, 727]",4,It is also recommended to stay hydrated and av...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,3,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...


In [68]:
all_qas.shape

(696, 10)

In [69]:
all_qas.head()

,origin_id,source_id,summary_id,model,questionId,sentId,sentence,predicate,question,answer
0,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,0,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,0,0,"Alfred Hitchcock once said "" Actors are cattle...",said,who said something?,Alfred Hitchcock
1,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,0,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,1,0,"Alfred Hitchcock once said "" Actors are cattle...",said,when did someone say something?,once
2,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,0,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,2,0,"Alfred Hitchcock once said "" Actors are cattle...",said,what did someone say?,Actors are cattle
0,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,1,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,0,0,"Alfred Hitchcock once said "" Actors are cattle...",said,who said something?,Alfred Hitchcock
1,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,1,608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b873...,1,0,"Alfred Hitchcock once said "" Actors are cattle...",said,when did someone say something?,once


In [72]:
all_qas[["origin_id", "sentId", "sentence", "question", "predicate", "answer"]].drop_duplicates().shape

(433, 6)

In [84]:
# remove duplicates because the same sentence can point to multiple sources and it's enough to verify each QA once 
all_qas[["origin_id", "sentId", "sentence", "question", "predicate", "answer"]].drop_duplicates().to_csv("../src/data/qasem_verification/batch7.csv", index=False)

# Update the QAs according to QASEm verification

In [48]:
# load annotation 
df = pd.read_csv("../src/data/qasem_verification/batch6_with_annotations.csv")
df.head()

,source_id,summary_id,model,questionId,sentId,sentence,predicate,question,answer,annotation
0,NPR-41366_Influence of Iran in Iraq,0,C,0,1,"They operate in Iraq , conducting military act...",operate,who operates somewhere?,They,Correct
1,NPR-41366_Influence of Iran in Iraq,0,C,1,1,"They operate in Iraq , conducting military act...",operate,where does someone operate?,in Iraq,Correct
2,NPR-41366_Influence of Iran in Iraq,0,C,2,1,"They operate in Iraq , conducting military act...",operate,how does someone operate somewhere?,conducting military activities and intelligenc...,Correct
3,NPR-41366_Influence of Iran in Iraq,0,C,3,1,"They operate in Iraq , conducting military act...",conducting,who is conducting something?,They,Correct
4,NPR-41366_Influence of Iran in Iraq,0,C,4,1,"They operate in Iraq , conducting military act...",conducting,where is someone conducting something?,in Iraq,Correct


In [49]:
df["annotation"].value_counts()

Correct          871
Innapropriate    179
Name: annotation, dtype: int64

In [50]:
'''
for topic in df, load json 
create new question id 
create mapping between old question Id and new question id 
remove questions 
remove spans 
'''


'\nfor topic in df, load json \ncreate new question id \ncreate mapping between old question Id and new question id \nremove questions \nremove spans \n'

In [51]:
df["source_id"].nunique()

18

In [52]:
groups = df.groupby("source_id")

In [9]:
source_id = 20
source_qas = groups.get_group(source_id)

KeyError: 20

In [39]:
def get_new_qa_id(qaIds, old_new_mapping):
    new_qaIds = []
    for qa_id in qaIds:
        if qa_id in old_new_mapping:
            new_qaIds.append(old_new_mapping[qa_id])

    return new_qaIds

In [53]:
output_dir = "../src/data/qasem_verification/v6"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [54]:
json_files_path = "../src/data/tofueval_chosen/mediasum"

In [55]:
for source_id, source_qas in tqdm(df.groupby("source_id"), total=df["source_id"].nunique()):
  with open(f"{json_files_path}/{source_id}.json", "r") as f:
      data = json.load(f)
  for summary_id, summary_qas in source_qas.groupby("summary_id"):
    # remove qas
    correct_qas = summary_qas[summary_qas["annotation"] == "Correct"].copy() # get only correct
    correct_qas['questionId'] = range(len(correct_qas)) # update qaId
    old_new_mapping_qa = dict(zip(summary_qas[summary_qas['annotation'] == 'Correct']['questionId'], correct_qas['questionId'])) # mapping from old to new qaID to update span id 
    df_qas = pd.DataFrame(data["summaries"][summary_id]["qas"]) 
    df_qas["questionId"] = df_qas["questionId"].apply(lambda x: old_new_mapping_qa.get(x, None))
    df_qas = df_qas.dropna().copy()
    df_qas["questionId"] = df_qas["questionId"].astype(int)
    data["summaries"][summary_id]["qas"] = df_qas.to_dict(orient="records")

    # remove spans 
    df_spans = pd.DataFrame(data["summaries"][summary_id]["spans"])
    df_spans["qaIds"] = df_spans["qaIds"].apply(lambda qaIds: get_new_qa_id(qaIds=qaIds, old_new_mapping=old_new_mapping_qa)) # remove spans from filtered qas 
    df_correct_spans = df_spans[df_spans["qaIds"].apply(len) > 0].copy() # update spans according to the filtered qas 
    df_correct_spans["id"] = range(len(df_correct_spans))
    old_new_mapping_span = dict(zip(df_spans[df_spans["qaIds"].apply(len) > 0]["id"], df_correct_spans["id"])) # mapping from old to new spanID to update predicates
    data["summaries"][summary_id]["spans"] = df_correct_spans.to_dict(orient="records")

    # update predicates 
    predicates = data["summaries"][summary_id]["predicates"]
    updated_predicates = []
    for pred in predicates:
        if pred in old_new_mapping_span: # remove predicate if all its QAs were filtered 
            updated_predicates.append(old_new_mapping_span[pred])
    data["summaries"][summary_id]["predicates"] = updated_predicates

    # remove spanIds in tokens 
    tokens = [{
      'id': token["id"],
      'text': token["text"],
      'lemma': token["lemma"],
      'spans': [],
      'class': token["class"],
      # 'label': token["label"]
    } for token in data["summaries"][summary_id]["tokens"]]
    data["summaries"][summary_id]["tokens"] = tokens 

  
  qa_num = len(data["summaries"][0]["qas"]) + len(data["summaries"][1]["qas"])
  print(f"source_id: {source_id}, QA: {qa_num}")
  with open(os.path.join(output_dir, f"{source_id}.json"), "w") as f:
     json.dump(data, f, indent=4)



  0%|          | 0/18 [00:00<?, ?it/s]

 22%|██▏       | 4/18 [00:00<00:00, 17.45it/s]

source_id: CNN-104129_Decline of American automobile industry, QA: 45
source_id: CNN-134470_Government Bailout, QA: 11
source_id: CNN-139946_Minnesota Senate Race, QA: 39
source_id: CNN-164885_Cuban celebration and government gathering, QA: 46
source_id: CNN-25553_Airlines' commitment to service improvements, QA: 56


 56%|█████▌    | 10/18 [00:00<00:00, 21.33it/s]

source_id: CNN-262980_Jeb Bush and Trump's immigration stance, QA: 51
source_id: CNN-340132_Trump administration's peace plan, QA: 40
source_id: CNN-388107_Shadow foreign policy, QA: 40
source_id: CNN-50484_Iraq's preparations for a potential US assault, QA: 57
source_id: CNN-62362_Illegal immigration, QA: 63


 72%|███████▏  | 13/18 [00:00<00:00, 21.50it/s]

source_id: CNN-7972_Federal Reserve's policy, QA: 51
source_id: NPR-18547_Problems with diaper changes for disabled adults, QA: 61
source_id: NPR-38350_Immigration debate, QA: 80
source_id: NPR-41366_Control and decision-making in Tehran, QA: 33
source_id: NPR-41366_Influence of Iran in Iraq, QA: 63


100%|██████████| 18/18 [00:00<00:00, 20.67it/s]

source_id: NPR-5615_Presidential helicopter cost, QA: 52
source_id: NPR-8010_Anti-Semitic incidents in schools, QA: 36
source_id: NPR-9336_Improvements in addressing cyberattacks, QA: 45
